In [50]:
# loading libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV as GridSearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve,roc_curve


In [51]:
features_gaby = pd.read_csv('../features/features_gaby.csv')
features_gonza = pd.read_csv('../features/features_gonza.csv').drop(columns='Unnamed: 0')
features_ari = pd.read_csv('../features/features_ariel.csv')
features_mati = pd.read_csv('../features/features_frecuencia.csv')
labels_training_set = pd.read_csv('../dataset/labels_training_set.csv')
test_set = pd.read_csv('../dataset/trocafone_kaggle_test.csv')
features_gonza2 = pd.read_csv('../features/features_gonza2.csv')

In [52]:
#model_df = pd.merge(features_gaby, features_gonza, how='inner', on='person')
model_df = pd.merge(features_gaby, features_ari, how='inner', on='person')
model_df = pd.merge(model_df, features_gonza, how='inner', on='person')
model_df = pd.merge(model_df, features_gonza2, how='inner', on='person')
#model_df = pd.merge(model_df, features_mati, how='inner', on='person')
model_df.head()

person  marca_favorita_apple  marca_favorita_asus  marca_favorita_lenovo  \
0  0008ed71                     1                    0                      0   
1  00091926                     1                    0                      0   
2  00091a7a                     1                    0                      0   
3  000ba417                     0                    0                      0   
4  000c79fe                     1                    0                      0   

   marca_favorita_lg  marca_favorita_motorola  marca_favorita_no tiene  \
0                  0                        0                        0   
1                  0                        0                        0   
2                  0                        0                        0   
3                  0                        0                        0   
4                  0                        0                        0   

   marca_favorita_quantum  marca_favorita_samsung  marca_favorita_sony  \
0                       0                       0                    0   
1                       0                       0                    0   
2                       0                       0                    0   
3                       0                       1                    0   
4                       0                       0                    0   

            ...             checkout  generic listing  searched products  \
0           ...                  3.0              1.0                NaN   
1           ...                  2.0              NaN                NaN   
2           ...                  NaN              NaN                NaN   
3           ...                  6.0             14.0                NaN   
4           ...                  1.0              1.0                9.0   

   viewed product  visited site  completa_flujo  favorite_device_Computer  \
0             NaN           2.0           False                         1   
1           372.0          34.0           False                         1   
2             3.0           1.0           False                         0   
3           153.0           6.0           False                         1   
4             3.0           1.0           False                         0   

   favorite_device_Smartphone  favorite_device_Tablet  favorite_device_Unknown  
0                           0                       0                        0  
1                           0                       0                        0  
2                           1                       0                        0  
3                           0                       0                        0  
4                           1                       0                        0  

[5 rows x 64 columns]

In [53]:
model_df.fillna(value=0, inplace=True)
training_set = pd.merge(model_df, labels_training_set, how='inner', on='person')
test_set = pd.merge(model_df, test_set, how='inner', on='person')
#training_set.drop(columns= 'Unnamed: 0', inplace=True)

In [54]:
features = [#'cant_total_eventos', #f1
            'marca_favorita_apple',#f2
            'mean_events_per_day',#f3
            #'has_converted_before',#f4
            #'most_viewed_sku',#f5
            #'most_viewed_color_Preto',#f6
            #'most_viewed_color_Branco',#f7
            #'most_viewed_color_Dourado', #f8
            #'region_brasil_Sao Paulo',#f9
            #'region_brasil_Unknown',#f10
            #'region_brasil_Minas Gerais',#f11
            'marca_favorita_samsung',#f12
            #'cant_ev_noche',#f13
            #'cant_ev_manana',#f14
            #'cant_ev_tarde',#f15
            #'cant_ev_madrugada',#f16
            #'cant_buenos',#f17
            #'cant_muy_buenos',#f18
            #'cant_excelentes',#f19
            #'nivel_de_actividad_Muy Bajo',#f20
            #'nivel_de_actividad_Muy Alto',#f21
            #'nivel_de_actividad_Bajo',#f22
            #'nivel_de_actividad_Medio',#f23
            #'nivel_de_actividad_Alto',#f24
            #'favorite_device_Smartphone',#f25
            'favorite_device_Computer',#f26
            'cantidad_checkouts',#f27
            'cantidad_conversions',#f28
            'tasa_cc',
            'viewed product',
            #'cantidad_lead',
            #'searched products',
            #'first_and_last_event_distance_in_days',
            #'month_moda',
            'completa_flujo'
           ]

In [112]:
x = np.array(training_set[features])
y = np.array(training_set['label'])
z = np.array(test_set[features])
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

In [56]:
def modelfit(alg):
    alg.fit(X_train, y_train, eval_metric='aucpr')
    pred = (alg.predict_proba(X_test)[:,1])
    accuracy = accuracy_score(y_test, pred.round(), normalize=False)

    print("Accuracy: ", accuracy / float(pred.size))

    for i in xrange(0,pred.size):
        if (pred[i] >= 0.5):
            pred[i] = 1
        else: 
            pred[i] = 0

    print('---Classification Report---')
    print(classification_report(y_test,pred))
    

In [69]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.8776719031676539)
---Classification Report---
             precision    recall  f1-score   support

          0       0.96      0.91      0.93      3699
          1       0.15      0.33      0.20       184

avg / total       0.93      0.88      0.90      3883



In [70]:
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18, seed=27), 
 param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=5)

gsearch1.fit(X_train, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.23813, std: 0.00827, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.23784, std: 0.00909, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.23781, std: 0.00641, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.24713, std: 0.01387, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.24851, std: 0.00983, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.25191, std: 0.00993, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.24786, std: 0.01413, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.25588, std: 0.01112, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.25226, std: 0.01876, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.22704, std: 0.00919, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.23347, std: 0.01035, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.24663, std: 0.01700, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 7, 'min_child_weight': 3

In [72]:
param_test2 = {
    'max_depth':[6,7,6],
    'min_child_weight':[2,3,4]
}

gsearch2 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18, seed=27), 
 param_grid = param_test2, scoring='f1',n_jobs=4,iid=False, cv=5)

gsearch2.fit(X_train, y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.24968, std: 0.00951, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.24702, std: 0.01202, params: {'max_depth': 6, 'min_child_weight': 3},
  mean: 0.25468, std: 0.01214, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.25320, std: 0.01593, params: {'max_depth': 7, 'min_child_weight': 2},
  mean: 0.25588, std: 0.01112, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.25612, std: 0.01667, params: {'max_depth': 7, 'min_child_weight': 4},
  mean: 0.24968, std: 0.00951, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.24702, std: 0.01202, params: {'max_depth': 6, 'min_child_weight': 3},
  mean: 0.25468, std: 0.01214, params: {'max_depth': 6, 'min_child_weight': 4}],
 {'max_depth': 7, 'min_child_weight': 4},
 0.25612001135528206)

In [73]:
param_test2b = {
    'min_child_weight':[4,5,6]
}

gsearch2 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18, seed=27), 
 param_grid = param_test2b, scoring='f1',n_jobs=4,iid=False, cv=5)

gsearch2.fit(X_train, y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.25612, std: 0.01667, params: {'min_child_weight': 4},
  mean: 0.25226, std: 0.01876, params: {'min_child_weight': 5},
  mean: 0.25558, std: 0.01727, params: {'min_child_weight': 6}],
 {'min_child_weight': 4},
 0.25612001135528206)

In [75]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=7,
 min_child_weight=4,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.8908060777749163)
---Classification Report---
             precision    recall  f1-score   support

          0       0.96      0.92      0.94      3699
          1       0.12      0.21      0.16       184

avg / total       0.92      0.89      0.90      3883



In [76]:
aram_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
 param_grid = param_test3, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.25612, std: 0.01667, params: {'gamma': 0.0},
  mean: 0.25319, std: 0.01327, params: {'gamma': 0.1},
  mean: 0.25185, std: 0.01329, params: {'gamma': 0.2},
  mean: 0.25992, std: 0.01513, params: {'gamma': 0.3},
  mean: 0.25641, std: 0.01641, params: {'gamma': 0.4}],
 {'gamma': 0.3},
 0.25991551769644705)

In [77]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=7,
 min_child_weight=3,
 gamma=0.3,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.8928663404584084)
---Classification Report---
             precision    recall  f1-score   support

          0       0.96      0.93      0.94      3699
          1       0.14      0.23      0.17       184

avg / total       0.92      0.89      0.91      3883



In [78]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gsearch4 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
 min_child_weight=3, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
 param_grid = param_test4, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.25768, std: 0.01199, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.25542, std: 0.01620, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.26177, std: 0.01515, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.25683, std: 0.01895, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.25646, std: 0.01072, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.25814, std: 0.00534, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.25398, std: 0.01626, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.25646, std: 0.01750, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.24773, std: 0.01391, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.25478, std: 0.01404, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.25982, std: 0.01284, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.26012, std: 0.01816, params: {'subsample': 0.9, 'colsample_bytree'

In [80]:
param_test5 = {
    'subsample':[i/100.0 for i in range(45,80,5)],
    'colsample_bytree':[i/100.0 for i in range(30,75,5)]
}

gsearch5 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
min_child_weight=3, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test5, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_train,y_train)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

([mean: 0.24466, std: 0.01174, params: {'subsample': 0.45, 'colsample_bytree': 0.3},
  mean: 0.24000, std: 0.01060, params: {'subsample': 0.5, 'colsample_bytree': 0.3},
  mean: 0.24070, std: 0.00544, params: {'subsample': 0.55, 'colsample_bytree': 0.3},
  mean: 0.24153, std: 0.00982, params: {'subsample': 0.6, 'colsample_bytree': 0.3},
  mean: 0.24378, std: 0.01392, params: {'subsample': 0.65, 'colsample_bytree': 0.3},
  mean: 0.24443, std: 0.00912, params: {'subsample': 0.7, 'colsample_bytree': 0.3},
  mean: 0.24664, std: 0.00536, params: {'subsample': 0.75, 'colsample_bytree': 0.3},
  mean: 0.25301, std: 0.01710, params: {'subsample': 0.45, 'colsample_bytree': 0.35},
  mean: 0.25022, std: 0.01480, params: {'subsample': 0.5, 'colsample_bytree': 0.35},
  mean: 0.25390, std: 0.00926, params: {'subsample': 0.55, 'colsample_bytree': 0.35},
  mean: 0.24994, std: 0.00919, params: {'subsample': 0.6, 'colsample_bytree': 0.35},
  mean: 0.24615, std: 0.00898, params: {'subsample': 0.65, 'colsam

In [89]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=7,
 min_child_weight=3,
 gamma=0.3,
 subsample=0.6,
 colsample_bytree=0.45,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.8776719031676539)
---Classification Report---
             precision    recall  f1-score   support

          0       0.96      0.91      0.93      3699
          1       0.14      0.30      0.19       184

avg / total       0.92      0.88      0.90      3883



In [90]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

gsearch6 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
min_child_weight=3, gamma=0.3, subsample=0.6, colsample_bytree=0.45,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test6, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train,y_train)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

([mean: 0.25626, std: 0.01702, params: {'reg_alpha': 1e-05},
  mean: 0.25920, std: 0.01421, params: {'reg_alpha': 0.01},
  mean: 0.26144, std: 0.01275, params: {'reg_alpha': 0.1},
  mean: 0.25641, std: 0.01145, params: {'reg_alpha': 1},
  mean: 0.23634, std: 0.00960, params: {'reg_alpha': 100}],
 {'reg_alpha': 0.1},
 0.26143842493851477)

In [96]:
param_test7 = {
    'reg_alpha':[0.08, 0.09, 0.1, 0.2, 1]
}

gsearch7 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
min_child_weight=3, gamma=0.3, subsample=0.6, colsample_bytree=0.45,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test7, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch7.fit(X_train,y_train)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch6.best_score_

([mean: 0.25349, std: 0.01759, params: {'reg_alpha': 0.08},
  mean: 0.26191, std: 0.01513, params: {'reg_alpha': 0.09},
  mean: 0.26144, std: 0.01275, params: {'reg_alpha': 0.1},
  mean: 0.25684, std: 0.01606, params: {'reg_alpha': 0.2},
  mean: 0.25641, std: 0.01145, params: {'reg_alpha': 1}],
 {'reg_alpha': 0.09},
 0.26143842493851477)

In [97]:
param_test8 = {
    'learning_rate':[0, 0.001, 0.01, 0.1, 0.2]
}

gsearch8 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
min_child_weight=3, gamma=0.3, subsample=0.6, colsample_bytree=0.45,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test8, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch8.fit(X_train,y_train)
gsearch8.grid_scores_, gsearch8.best_params_, gsearch8.best_score_

([mean: 0.00000, std: 0.00000, params: {'learning_rate': 0},
  mean: 0.26716, std: 0.01292, params: {'learning_rate': 0.001},
  mean: 0.26698, std: 0.01459, params: {'learning_rate': 0.01},
  mean: 0.25626, std: 0.01702, params: {'learning_rate': 0.1},
  mean: 0.23881, std: 0.01001, params: {'learning_rate': 0.2}],
 {'learning_rate': 0.001},
 0.2671625212684202)

In [98]:
param_test8 = {
    'learning_rate':[0, 0.0001, 0.001]
}

gsearch8 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
min_child_weight=3, gamma=0.3, subsample=0.6, colsample_bytree=0.45,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test8, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch8.fit(X_train,y_train)
gsearch8.grid_scores_, gsearch8.best_params_, gsearch8.best_score_

([mean: 0.00000, std: 0.00000, params: {'learning_rate': 0},
  mean: 0.26723, std: 0.01292, params: {'learning_rate': 0.0001},
  mean: 0.26716, std: 0.01292, params: {'learning_rate': 0.001}],
 {'learning_rate': 0.0001},
 0.26723255205681934)

In [108]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.0001,
 n_estimators=1000,
 max_depth=7,
 min_child_weight=3,
 gamma=0.3,
 subsample=0.6,
 colsample_bytree=0.45,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.8117434972959052)
---Classification Report---
             precision    recall  f1-score   support

          0       0.98      0.82      0.89      3699
          1       0.15      0.66      0.25       184

avg / total       0.94      0.81      0.86      3883



In [113]:
xgb1.fit(x,y, eval_metric='aucpr')
pred = (xgb1.predict_proba(z)[:,1])

In [115]:
persons = test_set['person']
submit = pd.DataFrame({'label': pred, 'person': persons})
submit = submit[['person', 'label']]
submit.to_csv('../submits/xgboost.csv', index=False)
no_conversiones = submit.loc[submit['label'] < 0.5]['label'].count()
print 'Cantidad de no conversiones: ' + str(no_conversiones)

conversiones = submit.loc[submit['label'] >= 0.5]['label'].count()
print 'Cantidad de conversiones: ' + str(conversiones)

Cantidad de no conversiones: 14985
Cantidad de conversiones: 4430


In [116]:
submit['label'].mean()

0.4895712733268738